# 01 - BaT Scraping

Scrape completed Porsche 911 auctions from Bring a Trailer.

**Outputs:**
- `data/raw/bat_listings.parquet` - Raw scraped data

In [1]:
# delete bad data if needed
# !rm data/raw/bat_listings.parquet

In [2]:
import logging
import pandas as pd
from pathlib import Path

from price_analysis.scraping import fetch_auctions, validate_scraped_data, DataQualityError
from price_analysis.scraping.bat import listings_to_dataframe

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Paths
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = RAW_DIR / "bat_listings.parquet"

## Configure Scraping

Adjust search queries and pagination as needed.

In [ ]:
# Search queries - multiple generation-specific searches to get more results
# BaT seems to limit how many results a single search returns
QUERIES = [
    "Porsche 996",  # 1999-2004 (996.1 and 996.2)
    "Porsche 997",  # 2005-2012 (997.1 and 997.2)
    "Porsche 991",  # 2012-2019 (991.1 and 991.2)
    "Porsche 992",  # 2020+ (992.1 and 992.2)
    "Porsche 911",  # Catch air-cooled + anything else missed
]

# Scraping parameters
MAX_CLICKS = 20  # Number of "Show More" button clicks (each loads ~24 listings)
DELAY = 1.0  # Seconds between requests (be polite!)
HEADLESS = True  # Set False to see browser for debugging

## Debug: Inspect BaT Page Structure (Optional)

Skip this section - fixture files already saved to `tests/fixtures/`.
Only re-run if BaT changes their DOM structure.

In [5]:
# Debug: Save sample pages for selector inspection
import time
from price_analysis.scraping import create_driver, save_debug_page

# Create driver (set HEADLESS=False above to watch)
driver = create_driver(headless=HEADLESS)

try:
    # 1. Fetch search results page
    search_url = "https://bringatrailer.com/auctions/results/?s=Porsche+911"
    logger.info(f"Fetching search page: {search_url}")
    driver.get(search_url)
    time.sleep(3)  # Wait for JS to load
    save_debug_page(driver, "bat_search_page", output_dir="../tests/fixtures")

    # 2. Fetch a specific listing page (992.1)
    listing_url = "https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/"
    logger.info(f"Fetching listing: {listing_url}")
    driver.get(listing_url)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_992", output_dir="../tests/fixtures")

    # 3. Fetch another listing (997.2) for variety
    listing_url_997 = "https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/"
    logger.info(f"Fetching listing: {listing_url_997}")
    driver.get(listing_url_997)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_997", output_dir="../tests/fixtures")

    logger.info("Debug pages saved to tests/fixtures/")

finally:
    driver.quit()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Fetching search page: https://bringatrailer.com/auctions/results/?s=Porsche+911
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_search_page.html
INFO:__main__:Fetching listing: https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_listing_992.html
INFO:__main__:Fetching listing: https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_listing_997.html
INFO:__main__:Debug pages saved to tests/fixtures/


## Run Scraper

This will take a while depending on MAX_PAGES. Each page + listing takes ~3-5 seconds.

In [ ]:
all_listings = []

for query in QUERIES:
    logger.info(f"Scraping: {query}")
    listings = fetch_auctions(
        query=query,
        max_clicks=MAX_CLICKS,
        delay=DELAY,
        headless=HEADLESS,
    )
    all_listings.extend(listings)
    logger.info(f"Found {len(listings)} listings for '{query}'")

logger.info(f"Total listings scraped: {len(all_listings)}")

INFO:__main__:Scraping: Porsche 996
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:price_analysis.scraping.bat:Fetching search results: https://bringatrailer.com/auctions/results/?s=Porsche+996
INFO:price_analysis.scraping.bat:Clicked 'Show More' (1/20)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (2/20)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (3/20)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (4/20)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (5/20)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (6/20)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (7/20)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (8/20)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (9/20)
INFO:price_analys

## Convert to DataFrame and Save

In [ ]:
df = listings_to_dataframe(all_listings)

# Run data quality checks and filter low-price listings (<$10k)
# Returns filtered DataFrame, raises DataQualityError if checks fail
df = validate_scraped_data(df)

display(df.head(10))
print(f"\nShape: {df.shape}")

INFO:price_analysis.scraping.bat:Filtered 13 listings below $10,000 (likely parts/salvage)
INFO:price_analysis.scraping.bat:Data quality checks passed: 112 listings, 112 with prices


,listing_url,title_raw,sale_price,sale_date,model_year,generation,trim,transmission,mileage,color,location
0,https://bringatrailer.com/listing/2003-porsche...,2003 Porsche 911 Carrera Targa 6-Speed,29250,2025-12-21,2003.0,996.2,Targa,Manual,51000.0,Arctic Silver Metallic,"Monrovia, California 91016"
1,https://bringatrailer.com/listing/2002-porsche...,2002 Porsche 911 Carrera 4S Coupe 6-Speed,28750,2025-10-22,2002.0,996.2,Carrera 4S,Manual,189000.0,Polar Silver Metallic,"Owings Mills, Maryland 21117"
3,https://bringatrailer.com/listing/2003-porsche...,2003 Porsche 911 Carrera Coupe,18750,2025-12-01,2003.0,996.2,Carrera,Automatic,101000.0,Midnight Blue Metallic,"Fort Lauderdale, Florida 33314"
4,https://bringatrailer.com/listing/2002-porsche...,2002 Porsche 911 Carrera 4S Coupe 6-Speed,36000,2025-10-14,2002.0,996.2,Carrera 4S,Manual,126000.0,Forest Green Metallic,"Lakeville, Connecticut 06039"
5,https://bringatrailer.com/listing/2003-porsche...,2003 Porsche 911 Carrera 4S Coupe 6-Speed,35996,2025-11-03,2003.0,996.2,Carrera 4S,Manual,78000.0,None,"Missouri City, Texas 77459"
6,https://bringatrailer.com/listing/1999-porsche...,39k-Mile 1999 Porsche 911 Carrera Cabriolet 6-...,29250,2025-10-10,1999.0,996.1,Carrera,Manual,39000.0,Black,"Pompano Beach, Florida 33064"
7,https://bringatrailer.com/listing/2004-porsche...,2004 Porsche 911 Carrera 4S Cabriolet 6-Speed,31000,2025-12-15,2004.0,996.2,Carrera 4S,Manual,82000.0,Guards Red,"Fort Lauderdale, Florida 33311"
9,https://bringatrailer.com/listing/2004-porsche...,2004 Porsche 911 Turbo Cabriolet,51000,2025-12-02,2004.0,996.2,Turbo,Tiptronic,69000.0,Black,"Huntington Beach, California 92648"
10,https://bringatrailer.com/listing/2004-porsche...,15k-Mile 2004 Porsche 911 Turbo Cabriolet X50 ...,72500,2025-10-30,2004.0,996.2,Turbo,Manual,15000.0,Black,"North Royalton, Ohio 44133"
11,https://bringatrailer.com/listing/1999-porsche...,1999 Porsche 911 Carrera Coupe 6-Speed,22500,2025-10-21,1999.0,996.1,Carrera,Manual,93000.0,Ocean Blue Metallic,"Dripping Springs, Texas 78620"



Shape: (112, 11)


In [ ]:
# Append to existing if present, otherwise create new
if OUTPUT_PATH.exists():
    existing = pd.read_parquet(OUTPUT_PATH)
    df = pd.concat([existing, df], ignore_index=True)
    df = df.drop_duplicates(subset=["listing_url"], keep="last")
    logger.info(f"Merged with existing data: {len(df)} total listings")

df.to_parquet(OUTPUT_PATH, index=False)
logger.info(f"Saved to {OUTPUT_PATH}")

INFO:__main__:Merged with existing data: 133 total listings
INFO:__main__:Saved to ../data/raw/bat_listings.parquet


## Quick Inspection

In [ ]:
print("Counts by generation:")
display(df["generation"].value_counts(dropna=False))

Counts by generation:


generation
996.2    57
996.1    51
None     14
997.1     5
991.2     2
992.1     2
992.2     1
997.2     1
Name: count, dtype: int64

In [ ]:
print("Counts by trim:")
display(df["trim"].value_counts(dropna=False))

Counts by trim:


trim
Carrera        52
Turbo          24
Carrera 4S     22
Carrera 4      12
None           10
Targa           4
GT3             4
Turbo S         2
Carrera S       1
GT3 Touring     1
GT2 RS          1
Name: count, dtype: int64

In [ ]:
print("Counts by transmission:")
display(df["transmission"].value_counts(dropna=False))

Counts by transmission:


transmission
Manual       113
Tiptronic      8
Automatic      7
None           3
PDK            2
Name: count, dtype: int64

In [ ]:
# Check parsing quality - how many have all required fields?
required = ["sale_price", "model_year", "generation", "trim", "transmission", "mileage"]
complete = df[required].notna().all(axis=1).sum()
print(
    f"\nListings with all required fields: {complete} / {len(df)} ({complete / len(df) * 100:.1f}%)"
)


Listings with all required fields: 112 / 133 (84.2%)


In [ ]:
# Sample some listings to verify parsing
print("Sample listings for manual verification:")
sample = df.sample(min(5, len(df)))
for _, row in sample.iterrows():
    print(f"\n{row['title_raw']}")
    print(
        f"  Parsed: {row['model_year']} {row['generation']} {row['trim']} ({row['transmission']})"
    )
    print(f"  Price: ${row['sale_price']:,}" if pd.notna(row["sale_price"]) else "  Price: N/A")
    print(f"  Mileage: {row['mileage']:,}" if pd.notna(row["mileage"]) else "  Mileage: N/A")

Sample listings for manual verification:

1995 Porsche 911 Carrera Coupe 6-Speed
  Parsed: 1995.0 None Carrera (Manual)
  Price: $137,000
  Mileage: 137,000.0

1999 Porsche 911 Carrera Coupe 6-Speed
  Parsed: 1999.0 996.1 Carrera (Manual)
  Price: $20,500
  Mileage: 139,000.0

36k-Mile 2000 Porsche 911 Carrera Coupe 6-Speed
  Parsed: 2000.0 996.1 Carrera (Manual)
  Price: $47,500
  Mileage: 36,000.0

LS3-Powered 1999 Porsche 911 Carrera 4 Coupe 6-Speed RWD
  Parsed: 1999.0 996.1 Carrera 4 (Manual)
  Price: $35,911
  Mileage: 187,000.0

RWD-Converted 2003 Porsche 911 Carrera 4S Coupe 6-Speed
  Parsed: 2003.0 996.2 Carrera 4S (Manual)
  Price: $135,000
  Mileage: 1,400.0
